# Export

Parse notebook and extract exportable code cells into corresponding TS modules (directives shamelessly copied from `nbdev`)

In [42]:
//| export

import { z } from "npm:zod@^3.23.8";
import path from "node:path";

## Configuration

`jurassic.json` contains project configuration

In [43]:
//| export

const configSchema: z.Schema = z.object({
  configPath: z.string(),
  nbsPath: z.string().default("."),
  outputPath: z.string().default("."),
});

export type Config = z.infer<typeof configSchema>;

🕵️‍♀️ Looking for config. Start from `cwd` and keep going up if needed looking for `jurassic.json`. When running notebooks, it seems like `cwd` points to notebook's directory (at least when running in VS Code). Hence this extra gymnastics, just to be on the safe side. Notice `d` (depth) and `maxD` (max depth) to make sure things don't get out of control 

In [44]:
//| export

const findConfig = async ( dir: string = Deno.cwd(), d = 0, config = "jurassic.json", maxD = 10): Promise<string> => {
  if (d >= maxD) { throw new Error("max depth reached"); }

  try {
    const f = path.join(dir, config);
    await Deno.lstat(f);
    return f;
  } catch {
    return findConfig(path.join(dir, "../"), d + 1);
  }
};

In [45]:
await findConfig();

"/Users/philip/projects/jurassic/jurassic.json"

Load and parse config. 

In [46]:
//| export

export const getConfig = async (): Promise<Config> => {
  const cf = await findConfig();
  const dcf = path.dirname(cf);
  const c = configSchema.parse(Object.assign({ configPath: cf }, JSON.parse(await Deno.readTextFile(cf))));
  c.nbsPath = path.join(dcf, c.nbsPath);
  c.outputPath = path.join(dcf, c.outputPath);
  return c;
};

In [47]:
await getConfig();

{
  configPath: "/Users/philip/projects/jurassic/jurassic.json",
  nbsPath: "/Users/philip/projects/jurassic/nbs",
  outputPath: "/Users/philip/projects/jurassic/jurassic"
}

Test helper to get a test config for a base directory. This is used internally to test setup in a temp directory.

In [48]:
const getTestConfig = (baseDir: string): Config =>
  configSchema.parse({
    configPath: path.resolve(baseDir, "jurassic.json"),
    nbsPath: path.join(baseDir, "nbs"),
    outputPath: path.join(baseDir, "jurassic"),
  });

In [49]:
getTestConfig("./")

{
  configPath: "/Users/philip/projects/jurassic/nbs/jurassic.json",
  nbsPath: "nbs",
  outputPath: "jurassic"
}

## Parse and process notebooks

Home made notebook parser + cell processor

In [50]:
//| export

const cellSchema = z.object({ cell_type: z.enum(["code", "markdown"]), source: z.array(z.string())  });
const nbSchema = z.object({ cells: z.array(cellSchema) });

type Cell = z.infer<typeof cellSchema>;
type Nb = z.infer<typeof nbSchema>;

Helpers for determining if a given line in a cell is a directive. Directives look like this:

```ts
//| export
```

In [51]:
//| export

const isDirective = (ln: string): boolean => ln.replaceAll(" ", "").startsWith("//|");


In [52]:
import { assertEquals } from "jsr:@std/assert";

Deno.test("isDirective", () => {
  assertEquals(isDirective("//| export"), true);
  assertEquals(isDirective("const c = 1;"), false);
  assertEquals(isDirective("// | export"), true);
  assertEquals(isDirective("// |    export"), true);
});


isDirective ... ok (0ms)

ok | 1 passed | 0 failed (0ms)


Determine if a given cell is exportable. "Exportable" means that its contents will end up in corresponding ts module.

In [53]:
//| export

const isCellExportable = (cell: Cell): boolean =>
  cell.cell_type === "code" &&
  cell.source.length > 0 &&
  isDirective(cell.source[0]) &&
  cell.source[0].includes("export");

In [54]:
import { assertEquals } from "jsr:@std/assert";

Deno.test("isCellExportable", () => {
  assertEquals(isCellExportable({ cell_type: "code", source: ["//| export\n"] }), true);
  assertEquals(isCellExportable({ cell_type: "code", source: ["const c = 1;"] }), false);
  assertEquals(isCellExportable({ cell_type: "code", source: ["//|export\n"] }), true);
  assertEquals(isCellExportable({ cell_type: "markdown", source: ["# showing //| export\n"] }), false);
  assertEquals(isCellExportable({ cell_type: "code", source: ["//|export"] }), true);
});


isCellExportable ... ok (0ms)

ok | 1 passed | 0 failed (0ms)


Process notebook - transfer exportable code from cells into ts module

In [55]:
//| export

const processNb = async (nbPath: string): Promise<string> => {
  const nb = nbSchema.parse(JSON.parse(await Deno.readTextFile(nbPath)));
  // we only need exportable cells
  const exportCells = nb.cells.filter((cell) => isCellExportable(cell));
  return exportCells.reduce(
    // get rid of directives, we want code only
    (acc, cell) => acc + cell.source.filter((s) => !isDirective(s)).join(""),
    `//source ${nbPath}\n\n`
  );
}

In [56]:
await Deno.jupyter.display(
  {
    "text/markdown": "```ts\n\n" + await processNb("./export.ipynb") + "\n```",
  },
  { raw: true }
);


```ts

//source ./export.ipynb


import { z } from "npm:zod@^3.23.8";
import path from "node:path";
const configSchema: z.Schema = z.object({
  configPath: z.string(),
  nbsPath: z.string().default("."),
  outputPath: z.string().default("."),
});

export type Config = z.infer<typeof configSchema>;
const findConfig = async ( dir: string = Deno.cwd(), d = 0, config = "jurassic.json", maxD = 10): Promise<string> => {
  if (d >= maxD) { throw new Error("max depth reached"); }

  try {
    const f = path.join(dir, config);
    await Deno.lstat(f);
    return f;
  } catch {
    return findConfig(path.join(dir, "../"), d + 1);
  }
};
export const getConfig = async (): Promise<Config> => {
  const cf = await findConfig();
  const dcf = path.dirname(cf);
  const c = configSchema.parse(Object.assign({ configPath: cf }, JSON.parse(await Deno.readTextFile(cf))));
  c.nbsPath = path.join(dcf, c.nbsPath);
  c.outputPath = path.join(dcf, c.outputPath);
  return c;
};
const cellSchema = z.object({ cell_type: z.enum(["code", "markdown"]), source: z.array(z.string())  });
const nbSchema = z.object({ cells: z.array(cellSchema) });

type Cell = z.infer<typeof cellSchema>;
type Nb = z.infer<typeof nbSchema>;
const isDirective = (ln: string): boolean => ln.replaceAll(" ", "").startsWith("//|");

const isCellExportable = (cell: Cell): boolean =>
  cell.cell_type === "code" &&
  cell.source.length > 0 &&
  isDirective(cell.source[0]) &&
  cell.source[0].includes("export");
const processNb = async (nbPath: string): Promise<string> => {
  const nb = nbSchema.parse(JSON.parse(await Deno.readTextFile(nbPath)));
  // we only need exportable cells
  const exportCells = nb.cells.filter((cell) => isCellExportable(cell));
  return exportCells.reduce(
    // get rid of directives, we want code only
    (acc, cell) => acc + cell.source.filter((s) => !isDirective(s)).join(""),
    `//source ${nbPath}\n\n`
  );
}
export const exportNb = async (notebookPath: string, config: Config): Promise<void> => {
  const fullPath = path.join(config.nbsPath, notebookPath);
  const fileInfo = await Deno.stat(fullPath);
  const notebooksToProcess: string[] = [];

  if (fileInfo.isDirectory) {
    // if target is a directory, let's go through all files/directories inside
    for await (const file of await Deno.readDir(fullPath)) {
      if (file.isDirectory) {
        // got another directory? delegate to another exportNb
        await exportNb(path.join(notebookPath, file.name), config);
        continue;
      }

      // we are only interested in notebooks
      if (!file.name.endsWith(".ipynb")) { continue; }

      // relative path only, puhleeze
      notebooksToProcess.push(
        path.relative(config.nbsPath, path.join(fullPath, file.name))
      );
    }
  }

  // let's go through all notebooks and process them one by one
  for (const notebook of notebooksToProcess) {
    // output module is the same as the input notebook, but with .ts extension
    const outputFile = notebook.replace(".ipynb", ".ts");
    // make sure we preserve subdirectories if any
    const outputDir = path.join(config.outputPath, path.dirname(outputFile));
    await Deno.mkdir(outputDir, { recursive: true });
    await Deno.writeTextFile(
      path.join(config.outputPath, outputFile),
      await processNb(path.resolve(config.nbsPath, notebook))
    );
  }
};

// create markdown representation of the directory listing files and subdirectories
export const dirListing = async (dir: string, d = 0): Promise<string> => {
  if (d > 10) {
    return "";
  }

  let md = "";
  for await (const f of Deno.readDir(dir)) {
    md += `${"  ".repeat(d)}- ${f.name}\n`;
    if (f.isDirectory) {
      md += await dirListing(path.join(dir, f.name), d + 1);
    }
  }
  return md;
};
```

Main export functionality. `exportNb` should work on both individual notebooks and directories containing notebooks and subdirectories containing more notebooks 🕳. `notebookPath` is relative to `config.nbsPath`

In [57]:
// | export

export const exportNb = async (notebookPath: string, config: Config): Promise<void> => {
  const fullPath = path.join(config.nbsPath, notebookPath);
  const fileInfo = await Deno.stat(fullPath);
  const notebooksToProcess: string[] = [];

  if (fileInfo.isDirectory) {
    // if target is a directory, let's go through all files/directories inside
    for await (const file of await Deno.readDir(fullPath)) {
      if (file.isDirectory) {
        // got another directory? delegate to another exportNb
        await exportNb(path.join(notebookPath, file.name), config);
        continue;
      }

      // we are only interested in notebooks
      if (!file.name.endsWith(".ipynb")) { continue; }

      // relative path only, puhleeze
      notebooksToProcess.push(
        path.relative(config.nbsPath, path.join(fullPath, file.name))
      );
    }
  }

  // let's go through all notebooks and process them one by one
  for (const notebook of notebooksToProcess) {
    // output module is the same as the input notebook, but with .ts extension
    const outputFile = notebook.replace(".ipynb", ".ts");
    // make sure we preserve subdirectories if any
    const outputDir = path.join(config.outputPath, path.dirname(outputFile));
    await Deno.mkdir(outputDir, { recursive: true });
    await Deno.writeTextFile(
      path.join(config.outputPath, outputFile),
      await processNb(path.resolve(config.nbsPath, notebook))
    );
  }
};


## Helpers

Random assorted helpers

In [58]:
//| export

// create markdown representation of the directory listing files and subdirectories
export const dirListing = async (dir: string, d = 0): Promise<string> => {
  if (d > 10) {
    return "";
  }

  let md = "";
  for await (const f of Deno.readDir(dir)) {
    md += `${"  ".repeat(d)}- ${f.name}\n`;
    if (f.isDirectory) {
      md += await dirListing(path.join(dir, f.name), d + 1);
    }
  }
  return md;
};

In [59]:
await Deno.jupyter.display({ "text/markdown": "```md\n" + (await dirListing("./")) + "\n```" }, { raw: true });


```md
- submodule
  - hello.ipynb
- export.ipynb

```

## Tests

Let's test export functionality

In [60]:
import { assert } from "jsr:@std/assert";

Deno.test("export", async (t) => {
  // set things up, let's recreate mini project structure inside a temp dir
  const td = await Deno.makeTempDir({});

  // recreate nbs dire in temp dir and copy notebooks there
  await Deno.mkdir(`${td}/nbs`), Deno.copyFileSync("./export.ipynb", `${td}/nbs/export.ipynb`);
  // recreate submodule directory and copy hello.ipynb to it
  await Deno.mkdir(`${td}/nbs/submodule`), Deno.copyFileSync("./submodule/hello.ipynb", `${td}/nbs/submodule/hello.ipynb`);

  await t.step("test export", async () => {
    await exportNb("./", getTestConfig(td));

    // make sure output modules are created
    const exportContent = await Deno.readTextFile(`${td}/jurassic/export.ts`);
    const submoduleExportContent = await Deno.readTextFile(`${td}/jurassic/submodule/hello.ts`);

    // spot check content inside the output modules
    assert(exportContent.includes("export const exportNb"));
    assert(submoduleExportContent.includes("export const foo"));

    // pretty print temp directory structure
    await Deno.jupyter.display({ "text/markdown": "```md\n" + (await dirListing(td)) + "\n```" }, { raw: true });
  });
});

export ...
  test export ...

```md
- nbs
  - submodule
    - hello.ipynb
  - export.ipynb
- jurassic
  - submodule
    - hello.ts
  - export.ts

```

 ok (3ms)
export ... ok (4ms)

ok | 1 passed (1 step) | 0 failed (4ms)


In [61]:
// run export on itself
await exportNb(".", await getConfig());